In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf

%matplotlib inline
import matplotlib

# 한글 폰트 설정
import matplotlib.pyplot as plt
import platform
if platform.system() == 'Darwin': # mac
        plt.rc('font', family='AppleGothic') 
else: # window or linux
        plt.rc('font', family='NanumGothicCoding') 
        
import warnings # warning message ignore
warnings.filterwarnings(action='ignore')

# gpu 사용 설정하기
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 
from tensorflow.python.client import device_lib
device_lib.list_local_devices() # cpu, gpu 여부 확인
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [2]:
# import zipfile

# local_zip = "./dataset.zip"
# zip_ref = zipfile.ZipFile(local_zip, 'r')
# zip_ref.extractall('./')
# zip_ref.close()

# train_zip = "./legal_docs_train.zip"
# zip_ref = zipfile.ZipFile(train_zip, 'r')
# zip_ref.extractall('./')


# test_zip = "./legal_docs_test.zip"
# zip_ref = zipfile.ZipFile(test_zip, 'r')
# zip_ref.extractall('./')
# zip_ref.close()

In [30]:
import json

with open("train.json", "r", encoding="utf-8-sig") as f:

    raw_json = json.load(f)

raw_json[0]

{'id': 196524,
 'extractive': [13, 14, 15],
 'article_original': ['[1] 의용 부동산등기법 제35조 제1항,',
  '제60조 제1항은 등기를 신청함에는 등기원인을 증명하는 서면을 제출하여야 하고,',
  '등기관리가 등기를 완료한 때에는 등기원인을 증명하는 서면',
  '또는 신청서 부본에 등기번호,',
  '신청서 수부(受附) 연월일,',
  '수부번호(受附番號),',
  '순위번호 및 등기제(登記濟)의 뜻을 기재하고,',
  '등기소의 인을 압날하여 이를 등기권리자에게 환부하여야 한다고 규정하고 있다.',
  '그러므로 매도증서에 위 규정에 따른 등기번호,',
  '등기순위, 등기제 등의 기재와 등기소인이 날인되어 있는 사실이 인정된다면,',
  '이는 등기 신청 시 등기원인을 증명하는 서면으로 제출되었다가 등기관리가 등기를 완료하고',
  '등기권리자에게 되돌려준 것으로 보지 않을 수 없다.',
  '따라서 특별한 사정이 없는 한 그 서면에 기재된 부동산에 관하여 그 기재의 등기번호와 순위번호에 따른 등기가 마쳐졌다고 인정하여야 한다.',
  '[2] 민사소송법 제356조 제1항은 문서의 작성방식과 취지에 의하여 공무원이 직무상 작성한 것으로 인정한 때에는 이를 진정한 공문서로 추정한다고 규정하고 있으나,',
  '위조 또는 변조 등 특별한 사정이 있다고 볼 만한 반증이 있는 경우에는 위와 같은 추정은 깨어진다.',
  '[3] 매도증서 등에 등기소의 등기제(登記濟)의 기재가 첨가됨으로써 사문서와 공문서로 구성된 문서는 공증에 관한 문서와는 달라 공문서 부분 성립이 인정된다고 하여 바로 사문서 부분인 매도증서 자체의 진정성립이 추정되거나 인정될 수는 없다.']}

In [29]:
dic = {"id":[], "extractive":[], "article_original":[]}

for raw in raw_json:
    dic["id"].append(raw["id"])
    dic["extractive"].append(raw["extractive"])
    dic["article_original"].append(raw["article_original"])
train_all = pd.DataFrame(dic)
train_all.head()

,id,extractive,article_original
0,196524,"[13, 14, 15]","[[1] 의용 부동산등기법 제35조 제1항,, 제60조 제1항은 등기를 신청함에는 ..."
1,106984,"[1, 2, 4]",[가. 주택개량을 위하여 조합원들이 스스로 결성한 주택개량재개발조합이 실시하는 재개...
2,190919,"[1, 2, 3]","[금원의 목적 내지 성질상 국가나 지방자치단체와 특정인 사이에서만 수수,, 결제되어..."
3,110573,"[0, 2, 5]",[가. 자동차대여업자의 직원으로서는 운전면허 없는 운전자가 위조된 운전면허증의 복사...
4,156698,"[0, 2, 3]",[소외회사의 평리사 6명을 제쳐 놓고 대표이사 3명만의 결의에 의하여 동회사의 대표...


In [31]:
with open("test.json", "r", encoding="utf-8-sig") as f:

    raw_json = json.load(f)

raw_json[0]

{'id': 79095,
 'article_original': ['승객이 놓고 내린 지하철의 전동차 바닥이나 선반 위에 있던 물건을 가지고 간 경우,',
  '지하철의 승무원은 유실물법상 전동차의 관수자로서 승객이 잊고 내린 유실물을 교부받을 권능을 가질 뿐 전동차 안에 있는 승객의 물건을 점유한다고 할 수 없고,',
  '그 유실물을 현실적으로 발견하지 않는 한 이에 대한 점유를 개시하였다고 할 수도 없으므로,',
  '그 사이에 위와 같은 유실물을 발견하고',
  '가져간 행위는 점유이탈물횡령죄에 해당함은 별론으로 하고',
  '절도죄에 해당하지는 않는다.']}

In [32]:
dic = {"id":[], "article_original":[]}

for raw in raw_json:
    dic["id"].append(raw["id"])
    dic["article_original"].append(raw["article_original"])
test_all = pd.DataFrame(dic)
test_all.head()

,id,article_original
0,79095,[승객이 놓고 내린 지하철의 전동차 바닥이나 선반 위에 있던 물건을 가지고 간 경우...
1,204506,[[1] 영업비밀 보유자가 거래 상대방에게 영업비밀을 사용하도록 승낙하는 의사표시는...
2,142079,[[1] 구 상속세 및 증여세법(2010. 1. 1. 법률 제9916호로 개정되기 ...
3,110816,[가. 처분문서는 그 진정성립이 인정되는 이상 법원은 그 기재내용을 부인할 만한 분...
4,207249,"[농지개량조합이 고용원을 일정기간 채용한 뒤 해임하고, 대부분 바로 그 다음날에 다..."
